In [2]:
import kagglehub
import os
import shutil
!pip install split-folders -q
!pip install ultralytics -q
import splitfolders
from ultralytics import YOLO

C:\Users\tanuj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\tanuj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\tanuj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\tanuj\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Download the Dataset & Copy Files to Desired Destination

In [3]:
# Download latest version
path = kagglehub.dataset_download("a2015003713/militaryaircraftdetectiondataset")

print("Path to dataset files:", path)

100%|██████████| 10.1G/10.1G [08:05<00:00, 22.4MB/s]  

Extracting files...


Path to dataset files: C:\Users\tanuj\.cache\kagglehub\datasets\a2015003713\militaryaircraftdetectiondataset\versions\90


In [4]:
source_dir = f'{path}/crop/'
destination_dir = '/content/dataset/'
subfolders_to_copy = ['F15', 'F16', 'F18']

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Copy each specified subfolder
for folder in subfolders_to_copy:
    source_path = os.path.join(source_dir, folder)
    destination_path = os.path.join(destination_dir, folder)
    if os.path.exists(source_path):
        shutil.copytree(source_path, destination_path)
        print(f"Copied {folder} to {destination_dir}")
    else:
        print(f"Source folder {folder} not found in {source_dir}")
        raise FileNotFoundError

print("Copying complete.")

Copied F15 to /content/dataset/
Copied F16 to /content/dataset/
Copied F18 to /content/dataset/
Copying complete.


# Label Files & Transfer Images to a Single Folder

In [5]:
aircrafts = ["F15", "F16", "F18"]
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

for i, aircraft in enumerate(aircrafts):

  image_directory = f"/content/dataset/{aircraft}"
  label_directory = "/content/dataset/labels"
  CLASS_ID = i

  os.makedirs(label_directory, exist_ok=True)

  yolo_label_content = f"{CLASS_ID} 0.5 0.5 1.0 1.0"

  print(f"▶️ Reading images from: {image_directory}")
  print("-" * 30)

  if not os.path.isdir(image_directory):
      continue
  else:
      for filename in os.listdir(image_directory):
          if os.path.splitext(filename)[1].lower() in image_extensions:
              label_filename = os.path.splitext(filename)[0] + ".txt"
              label_filepath = os.path.join(label_directory, label_filename)
              try:
                  with open(label_filepath, 'w') as f:
                      f.write(yolo_label_content)
              except IOError as e:
                  print(f"❌ Error creating label for {filename}: {e}")

▶️ Reading images from: /content/dataset/F15
------------------------------
▶️ Reading images from: /content/dataset/F16
------------------------------
▶️ Reading images from: /content/dataset/F18
------------------------------


In [6]:
# Create the destination directory for all images if it doesn't exist
all_images_destination_dir = '/content/dataset/images/'
os.makedirs(all_images_destination_dir, exist_ok=True)


print("Starting image transfer to a single folder...")
for aircraft in aircrafts:
    source_image_directory = f"/content/dataset/{aircraft}"
    print(f"Processing images from: {source_image_directory}")
    if os.path.isdir(source_image_directory):
        for filename in os.listdir(source_image_directory):
            if os.path.splitext(filename)[1].lower() in image_extensions:
                source_filepath = os.path.join(source_image_directory, filename)
                destination_filepath = os.path.join(all_images_destination_dir, filename)
                try:
                    # Use shutil.move to move the file
                    shutil.move(source_filepath, destination_filepath)
                    # print(f"Moved {filename} to {all_images_destination_dir}")
                except IOError as e:
                    print(f"❌ Error moving {filename}: {e}")
    else:
        print(f"Source directory not found: {source_image_directory}")

print("Image transfer complete.")

Starting image transfer to a single folder...
Processing images from: /content/dataset/F15
Processing images from: /content/dataset/F16
Processing images from: /content/dataset/F18
Image transfer complete.


In [7]:
for aircraft in aircrafts:
    folder_path = f"/content/dataset/{aircraft}"
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        shutil.rmtree(folder_path)
        print(f"Deleted folder: {folder_path}")
    else:
        print(f"Folder not found or not a directory: {folder_path}")

Deleted folder: /content/dataset/F15
Deleted folder: /content/dataset/F16
Deleted folder: /content/dataset/F18


# Preparing the Dataset

In [8]:
input_folder = '/content/dataset/'

output_folder = '/content/split/'

splitfolders.ratio(input_folder,
                   output=output_folder,
                   seed=42,
                   ratio=(.8, .2),
                   group_prefix=None,
                   move=False) # Set to True to move files instead of copying

print("Dataset splitting complete!")

Copying files: 9990 files [00:46, 215.37 files/s]

Dataset splitting complete!


In [9]:
import yaml

dataset_yaml = '/content/dataset.yaml'
data = {
        "train": "/content/split/train/images",
        "val": "/content/split/val/images",
        "names":  ["F15", "F16", "F18"],
        "nc": 3
    }

try:
    with open(dataset_yaml, 'w') as f:
        yaml.dump(data, f, default_flow_style=False)
except IOError as e:
    print(f"❌ Error creating label for {dataset_yaml}: {e}")


Create this dataset.yaml:

train: /content/split/train/images

val: /content/split/val/images

nc: 9

names: ["F15", "F16", "F18", "F22", "F35", "B1", "B2", "C17", "C130"]

# Fine-Tune the Model

## Test Run

In [10]:
model = YOLO('yolo11n.pt')
dataset_yaml = '/content/dataset.yaml'

try:
    results = model.train(
        data=dataset_yaml,
        epochs=3,
        imgsz=640,
        batch=8,
        save=False,
        plots=False
    )
except Exception as e:
    print(f"An error occurred during training: {e}")

100%|██████████| 5.35M/5.35M [00:00<00:00, 18.2MB/s]


Ultralytics 8.3.165  Python-3.11.9 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-1334U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=False, pose=12.0, pretrained=T

100%|██████████| 755k/755k [00:00<00:00, 8.76MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning C:\content\split\train\labels... 3989 images, 0 backgrounds, 7 corrupt: 100%|██████████| 3996/3996 [00:07<00:00, 501.81it/s]

...

                   all        996        996      0.329      0.774      0.396      0.331

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        3/3         0G     0.1139      1.132      0.871         18        640: 100%|██████████| 499/499 [37:39<00:00,  4.53s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [02:01<00:00,  1.92s/it]


                   all        996        996      0.408      0.808      0.531      0.529

3 epochs completed in 2.234 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 5.4MB
Optimizer stripped from runs\detect\train\weights\best.pt, 5.4MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.165  Python-3.11.9 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-1334U)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [01:33<00:00,  1.49s/it]


                   all        996        996      0.407      0.808      0.531      0.529
                   F15        310        310       0.36      0.726      0.387      0.384
                   F16        359        359      0.468      0.794      0.627      0.626
                   F18        327        327      0.394      0.905      0.578      0.578
Speed: 1.3ms preprocess, 76.5ms inference, 0.0ms loss, 2.7ms postprocess per image


## Training the model with various learning rates

In [11]:

results = model.train(
    data=dataset_yaml,
    #cfg=best_hyp_yaml,
    epochs=4,
    imgsz=640,
    batch=16, lr0=0.01, name="run_lr_0_01"
)

Ultralytics 8.3.165  Python-3.11.9 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-1334U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=4, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run_lr_0_01, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=False, pose=12.0, pretr

train: Scanning C:\content\split\train\labels.cache... 3989 images, 0 backgrounds, 7 corrupt: 100%|██████████| 3996/3996 [00:00<?, ?it/s]

train: C:\content\split\train\images\2957729725f0acb08acb86af5e28c5f3_0.jpg: ignoring corrupt image/label: image size (9, 20) <10 pixels
train: C:\content\split\train\images\8a0b8bad3da60ee3b07c0c7b5cd65213_1.jpg: ignoring corrupt image/label: image size (7, 18) <10 pixels
train: C:\content\split\train\images\a9fe7dbcb36994e93ecc56919a5be755_1.jpg: ignoring corrupt image/label: image size (7, 11) <10 pixels
train: C:\content\split\train\images\b9c46d9092ddb799c80855e16e1af793_1.jpg: ignoring corrupt image/label: image size (9, 18) <10 pixels
train: C:\content\split\train\images\bae97f090c5d13bf4f5557c82d6d2d4a_0.jpg: ignoring corrupt image/label: image size (9, 21) <10 pixels
train: C:\content\split\train\images\f31ec6705c27e98c716ce6c68d44ca76_1.jpg: ignoring corrupt image/label: image size (9, 30) <10 pixels
train: C:\content\split\train\images\ff36b73cf344b8634663df1bb37f2a62_1.jpg: ignoring corrupt image/label: image size (9, 16) <10 pixels
val: Fast image access  (ping: 0.10.1 ms,


C:\Users\tanuj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\content\split\val\labels.cache... 996 images, 0 backgrounds, 3 corrupt: 100%|██████████| 999/999 [00:00<?, ?it/s]

val: C:\content\split\val\images\a9fe7dbcb36994e93ecc56919a5be755_0.jpg: ignoring corrupt image/label: image size (7, 11) <10 pixels
val: C:\content\split\val\images\e511decc1e28e3f1c41ec70ba86110f5_1.jpg: ignoring corrupt image/label: image size (9, 20) <10 pixels
val: C:\content\split\val\images\ff36b73cf344b8634663df1bb37f2a62_0.jpg: ignoring corrupt image/label: image size (9, 18) <10 pixels
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\run_lr_0_01
Starting training for 4 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


        4/4         0G     0.0844     0.9244     0.8651         16        640: 100%|██████████| 250/250 [34:15<00:00,  8.22s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [01:41<00:00,  3.19s/it]

                   all        996        996      0.586      0.781      0.742      0.735



4 epochs completed in 2.965 hours.
Optimizer stripped from runs\detect\run_lr_0_01\weights\last.pt, 5.4MB
Optimizer stripped from runs\detect\run_lr_0_01\weights\best.pt, 5.4MB

Validating runs\detect\run_lr_0_01\weights\best.pt...
Ultralytics 8.3.165  Python-3.11.9 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-1334U)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [01:24<00:00,  2.65s/it]

                   all        996        996      0.592      0.775      0.742      0.736
                   F15        310        310      0.459      0.819       0.65      0.635
                   F16        359        359      0.519      0.947       0.79      0.787
                   F18        327        327      0.798       0.56      0.786      0.785
Speed: 1.3ms preprocess, 69.9ms inference, 0.0ms loss, 1.0ms postprocess per image


In [12]:
results = model.train(
    data=dataset_yaml,
    #cfg=best_hyp_yaml,
    epochs=4,
    imgsz=640,
    batch=16, lr0=0.1, name="run_lr_0_1"
)

Ultralytics 8.3.165  Python-3.11.9 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-1334U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=4, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.1, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run_lr_0_1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=False, pose=12.0, pretrai

train: Scanning C:\content\split\train\labels.cache... 3989 images, 0 backgrounds, 7 corrupt: 100%|██████████| 3996/3996 [00:00<?, ?it/s]

train: C:\content\split\train\images\2957729725f0acb08acb86af5e28c5f3_0.jpg: ignoring corrupt image/label: image size (9, 20) <10 pixels
train: C:\content\split\train\images\8a0b8bad3da60ee3b07c0c7b5cd65213_1.jpg: ignoring corrupt image/label: image size (7, 18) <10 pixels
train: C:\content\split\train\images\a9fe7dbcb36994e93ecc56919a5be755_1.jpg: ignoring corrupt image/label: image size (7, 11) <10 pixels
train: C:\content\split\train\images\b9c46d9092ddb799c80855e16e1af793_1.jpg: ignoring corrupt image/label: image size (9, 18) <10 pixels
train: C:\content\split\train\images\bae97f090c5d13bf4f5557c82d6d2d4a_0.jpg: ignoring corrupt image/label: image size (9, 21) <10 pixels
train: C:\content\split\train\images\f31ec6705c27e98c716ce6c68d44ca76_1.jpg: ignoring corrupt image/label: image size (9, 30) <10 pixels
train: C:\content\split\train\images\ff36b73cf344b8634663df1bb37f2a62_1.jpg: ignoring corrupt image/label: image size (9, 16) <10 pixels
val: Fast image access  (ping: 0.10.0 ms,


C:\Users\tanuj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\content\split\val\labels.cache... 996 images, 0 backgrounds, 3 corrupt: 100%|██████████| 999/999 [00:00<?, ?it/s]

val: C:\content\split\val\images\a9fe7dbcb36994e93ecc56919a5be755_0.jpg: ignoring corrupt image/label: image size (7, 11) <10 pixels
val: C:\content\split\val\images\e511decc1e28e3f1c41ec70ba86110f5_1.jpg: ignoring corrupt image/label: image size (9, 20) <10 pixels
val: C:\content\split\val\images\ff36b73cf344b8634663df1bb37f2a62_0.jpg: ignoring corrupt image/label: image size (9, 18) <10 pixels
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.1' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 


...


        4/4         0G     0.0772     0.7854     0.8632         16        640: 100%|██████████| 250/250 [20:43<00:00,  4.97s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [01:02<00:00,  1.94s/it]

                   all        996        996      0.707      0.778      0.843      0.828



4 epochs completed in 2.450 hours.
Optimizer stripped from runs\detect\run_lr_0_1\weights\last.pt, 5.4MB
Optimizer stripped from runs\detect\run_lr_0_1\weights\best.pt, 5.4MB

Validating runs\detect\run_lr_0_1\weights\best.pt...
Ultralytics 8.3.165  Python-3.11.9 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-1334U)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:53<00:00,  1.67s/it]

                   all        996        996      0.707      0.776      0.843      0.828
                   F15        310        310      0.579      0.839      0.794       0.76
                   F16        359        359      0.721      0.852      0.879      0.876
                   F18        327        327      0.823      0.638      0.855      0.849
Speed: 0.9ms preprocess, 47.9ms inference, 0.0ms loss, 0.4ms postprocess per image


In [13]:
results = model.train(
    data=dataset_yaml,
    #cfg=best_hyp_yaml,
    epochs=4,
    imgsz=640,
    batch=16, lr0=0.001, name="run_lr_0_001"
)

Ultralytics 8.3.165  Python-3.11.9 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-1334U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=4, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run_lr_0_001, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=False, pose=12.0, pre

train: Scanning C:\content\split\train\labels.cache... 3989 images, 0 backgrounds, 7 corrupt: 100%|██████████| 3996/3996 [00:00<?, ?it/s]

train: C:\content\split\train\images\2957729725f0acb08acb86af5e28c5f3_0.jpg: ignoring corrupt image/label: image size (9, 20) <10 pixels
train: C:\content\split\train\images\8a0b8bad3da60ee3b07c0c7b5cd65213_1.jpg: ignoring corrupt image/label: image size (7, 18) <10 pixels
train: C:\content\split\train\images\a9fe7dbcb36994e93ecc56919a5be755_1.jpg: ignoring corrupt image/label: image size (7, 11) <10 pixels
train: C:\content\split\train\images\b9c46d9092ddb799c80855e16e1af793_1.jpg: ignoring corrupt image/label: image size (9, 18) <10 pixels
train: C:\content\split\train\images\bae97f090c5d13bf4f5557c82d6d2d4a_0.jpg: ignoring corrupt image/label: image size (9, 21) <10 pixels
train: C:\content\split\train\images\f31ec6705c27e98c716ce6c68d44ca76_1.jpg: ignoring corrupt image/label: image size (9, 30) <10 pixels
train: C:\content\split\train\images\ff36b73cf344b8634663df1bb37f2a62_1.jpg: ignoring corrupt image/label: image size (9, 16) <10 pixels
val: Fast image access  (ping: 0.00.0 ms,


C:\Users\tanuj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\content\split\val\labels.cache... 996 images, 0 backgrounds, 3 corrupt: 100%|██████████| 999/999 [00:00<?, ?it/s]

val: C:\content\split\val\images\a9fe7dbcb36994e93ecc56919a5be755_0.jpg: ignoring corrupt image/label: image size (7, 11) <10 pixels
val: C:\content\split\val\images\e511decc1e28e3f1c41ec70ba86110f5_1.jpg: ignoring corrupt image/label: image size (9, 20) <10 pixels
val: C:\content\split\val\images\ff36b73cf344b8634663df1bb37f2a62_0.jpg: ignoring corrupt image/label: image size (9, 18) <10 pixels


...


        4/4         0G    0.07369     0.6986     0.8617         16        640: 100%|██████████| 250/250 [25:24<00:00,  6.10s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [01:06<00:00,  2.09s/it]

                   all        996        996      0.738      0.782      0.843      0.817



4 epochs completed in 1.681 hours.
Optimizer stripped from runs\detect\run_lr_0_001\weights\last.pt, 5.4MB
Optimizer stripped from runs\detect\run_lr_0_001\weights\best.pt, 5.4MB

Validating runs\detect\run_lr_0_001\weights\best.pt...
Ultralytics 8.3.165  Python-3.11.9 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-1334U)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:55<00:00,  1.74s/it]

                   all        996        996      0.738       0.78      0.843      0.816
                   F15        310        310      0.621      0.816      0.781      0.734
                   F16        359        359      0.858      0.706      0.882       0.86
                   F18        327        327      0.734      0.817      0.867      0.855
Speed: 0.9ms preprocess, 49.9ms inference, 0.0ms loss, 0.4ms postprocess per image


## Validation

In [17]:
from ultralytics import YOLO
import glob
import os

# Try to find the best.pt file in available runs
# Update this path to where your runs are actually saved
run_dirs = glob.glob('C:/Users/tanuj/Downloads/runs/detect/*/weights/best.pt')
if not run_dirs:
	raise FileNotFoundError("No 'best.pt' file found in any run directory. Please check your training runs path.")
MODEL_PATH = run_dirs[0]  # Use the first found model, or select as needed

dataset_yaml = '/content/dataset.yaml'
model = YOLO(MODEL_PATH)
results = model.val(data=dataset_yaml, plots=True)

Ultralytics 8.3.165  Python-3.11.9 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-1334U)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 902.8894.6 MB/s, size: 278.4 KB)


val: Scanning C:\content\split\val\labels.cache... 996 images, 0 backgrounds, 3 corrupt: 100%|██████████| 999/999 [00:00<?, ?it/s]

val: C:\content\split\val\images\a9fe7dbcb36994e93ecc56919a5be755_0.jpg: ignoring corrupt image/label: image size (7, 11) <10 pixels
val: C:\content\split\val\images\e511decc1e28e3f1c41ec70ba86110f5_1.jpg: ignoring corrupt image/label: image size (9, 20) <10 pixels
val: C:\content\split\val\images\ff36b73cf344b8634663df1bb37f2a62_0.jpg: ignoring corrupt image/label: image size (9, 18) <10 pixels



C:\Users\tanuj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [01:01<00:00,  1.03it/s]


                   all        996        996      0.733      0.776      0.842      0.815
                   F15        310        310      0.619       0.81      0.776      0.728
                   F16        359        359      0.857      0.699      0.879      0.859
                   F18        327        327      0.724       0.82       0.87      0.857
Speed: 0.8ms preprocess, 51.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val


In [21]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Use run_dirs to get the correct experiment folders and learning rates
run_dir_map = {
    'run_lr_0_001': 0.001,
    'run_lr_0_01': 0.01,
    'run_lr_0_1': 0.1
}

metrics = []

for run_path in run_dirs:
    # Extract experiment name from the path
    exp_name = os.path.basename(os.path.dirname(os.path.dirname(run_path)))
    if exp_name not in run_dir_map:
        continue
    lr = run_dir_map[exp_name]
    exp_dir = os.path.dirname(os.path.dirname(run_path))
    csv_path = os.path.join(exp_dir, 'results.csv')
    if not os.path.exists(csv_path):
        print(f"results.csv not found for {exp_name} at {csv_path}")
        continue
    df_temp = pd.read_csv(csv_path)
    best_idx = df_temp['metrics/mAP50(B)'].idxmax()
    row = df_temp.loc[best_idx]

    metrics.append({
        'Learning Rate': lr,
        'mAP@0.5': row['metrics/mAP50(B)'],
        'mAP@0.5:0.95': row['metrics/mAP50-95(B)'],
        'Precision': row['metrics/precision(B)'],
        'Recall': row['metrics/recall(B)']
    })

df = pd.DataFrame(metrics)
print(df)


   Learning Rate  mAP@0.5  mAP@0.5:0.95  Precision   Recall
0          0.001  0.84318       0.81699    0.73797  0.78198
1          0.010  0.74233       0.73540    0.58604  0.78137
2          0.100  0.84284       0.82820    0.70673  0.77843


In [22]:
# Plot
plt.figure(figsize=(10, 6))
plt.plot(df['Learning Rate'], df['mAP@0.5'], label='mAP@0.5', marker='o')
plt.plot(df['Learning Rate'], df['mAP@0.5:0.95'], label='mAP@0.5:0.95', marker='o')
plt.plot(df['Learning Rate'], df['Precision'], label='Precision', marker='o')
plt.plot(df['Learning Rate'], df['Recall'], label='Recall', marker='o')

plt.xscale('log')  # Optional: log scale makes learning rates easier to visualize
plt.xlabel('Learning Rate')
plt.ylabel('Score')
plt.title('YOLO Validation Metrics vs Learning Rate')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 1 Axes>

# Saving the Work

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This creates a project folder in your Drive and copies the runs folder into it
!mkdir -p /content/drive/MyDrive/YOLO_Aircraft_Project/
!cp -r /content/runs /content/drive/MyDrive/YOLO_Aircraft_Project/